In [58]:
import sys
sys.path.append("src")

In [59]:
%pwd

'c:\\Users\\reo\\OneDrive - The University of Tokyo\\github'

In [60]:
import os
print(os.listdir())

['demo.ipynb', 'environment.yml', 'Figure_1.png', 'notebools', 'requirements.txt', 'src', '__pycache__']


In [65]:
from network import build_vonmises_weights
W = build_vonmises_weights()
print("W min/max:", W.min(), W.max())
import numpy as np

row_sums = W.sum(axis=1)
print("row_sums min:", row_sums.min())
print("row_sums max:", row_sums.max())
tol = 1e-6
if np.all(np.abs(row_sums) < tol):
    print("OK: all row sums ≃ 0 within tol=", tol)
else:
    bad_idxs = np.where(np.abs(row_sums) >= tol)[0]
    print(f"WARNING: {len(bad_idxs)} rows exceed tol:", bad_idxs[:10])

W min/max: -11.283606313247597 43.29622808100791
row_sums min: -2.4158453015843406e-13
row_sums max: 1.652011860642233e-13
OK: all row sums ≃ 0 within tol= 1e-06


In [68]:
# 例: 多くのニューロンが発火している時点を探す
activity = records.sum(axis=1)        # 各時刻の全ニューロン合計発火量
t_peak = np.argmax(activity)          # 一番活動が高い時刻
print("Peak activity at t =", t_peak)

spikes    = records[t_peak]
occupancy = np.ones_like(spikes)

Peak activity at t = 340


In [69]:
import sys, os
sys.path.insert(0, os.path.join(os.getcwd(), "src"))

from simulate import run_simulation
import matplotlib.pyplot as plt
from config import dt

# シミュレーション再実行（安定パラメータで）
theta, records = run_simulation(noise_sigma=0.01, bump_center_speed=0.005)
print(records.max())
"""
# ヒートマップ
plt.figure(figsize=(8,3))
plt.imshow(records.T, aspect='auto', origin='lower',
           extent=[0, records.shape[0]*dt, 0, 360],
           vmin=0, vmax=records.max()*0.8)
plt.xlabel("Time (ms)")
plt.ylabel("Neuron angle (deg)")
plt.title("Activity Heatmap")
plt.colorbar(label="Firing rate")
plt.show()
"""

nan


'\n# ヒートマップ\nplt.figure(figsize=(8,3))\nplt.imshow(records.T, aspect=\'auto\', origin=\'lower\',\n           extent=[0, records.shape[0]*dt, 0, 360],\n           vmin=0, vmax=records.max()*0.8)\nplt.xlabel("Time (ms)")\nplt.ylabel("Neuron angle (deg)")\nplt.title("Activity Heatmap")\nplt.colorbar(label="Firing rate")\nplt.show()\n'

In [ ]:
import numpy as np

print("rate_smooth min/max:", np.nanmin(rate_smooth), np.nanmax(rate_smooth))
print("NaN count:", np.isnan(rate_smooth).sum())
print("Inf count:", np.isinf(rate_smooth).sum())

In [ ]:
bad_idx = np.where(~np.isfinite(rate_smooth))[0]
print("Bad bins:", bad_idx)


In [ ]:
print(records.max())

In [ ]:
from analysis.tuning import compute_tuning, fit_vonmises
from viz import plot_tuning_curve

rate_smooth = compute_tuning(spikes, theta, occupancy, sigma=6)
# 前処理：NaN/Inf をクリップ
rate_smooth = np.nan_to_num(rate_smooth, nan=0.0, posinf=np.nanmax(rate_smooth))
rate_smooth = np.clip(rate_smooth, 0, np.nanpercentile(rate_smooth, 99))

# von Mises フィット
valid = np.isfinite(rate_smooth)
mu, kappa, A, B = fit_vonmises(rate_smooth[valid], theta[valid])
print(f"μ={np.degrees(mu):.1f}°, κ={kappa:.2f}")

# プロット
plot_tuning_curve(theta, rate_smooth, params=(mu, kappa, A, B))